In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import torch

In [28]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/anna-
[nltk_data]     alexandradanchenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
RANDOM_SEED = 42

# Import Data

In [30]:
job_level_data = pd.read_csv("../data/JobLevelData_preproc.csv")
classes_labels = ['Chief Officer', 'Director', 'Individual Contributor/Staff', 'Manager',
 'Vice President']
job_level_data

,Title,Label
0,Vice President Director of Systems Engineering,"[0, 0, 0, 0, 1]"
1,Systems Engineer Systems Architect,"[0, 0, 1, 1, 0]"
2,Executive Director Global Informational Techno...,"[1, 1, 0, 0, 0]"
3,Chief Technology Officer Executive Director of...,"[1, 1, 0, 0, 0]"
4,Vice President Chief Information Officer,"[0, 0, 0, 0, 1]"
...,...,...
2221,Net Software Architect and Team Project Lead,"[0, 0, 0, 1, 0]"
2222,Solutions Architect Technical Lead,"[0, 0, 1, 1, 0]"
2223,Manager Salesforcecom Administration and Relea...,"[0, 0, 0, 1, 0]"
2224,Innovation Automation Architect,"[0, 0, 0, 1, 0]"


To establish a baseline model for this multi label text classification task, it was chosen to transform the model to binary classification problem. This involves dividing the problem into 5 binary classification problems, one for each class.

In [31]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = ' '.join([word for word in word_tokens if word.lower() not in stop_words])
    return filtered_text

job_level_data["Title"] = job_level_data["Title"].apply(remove_stopwords)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    job_level_data['Title'],
    job_level_data['Label'],
    test_size=0.2,
    random_state=RANDOM_SEED,
    shuffle=True
)

In [32]:
tfidf = TfidfVectorizer(ngram_range=(2,2))
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

X_train = pd.DataFrame(X_train.toarray(), columns=tfidf.get_feature_names_out())
X_test = pd.DataFrame(X_test.toarray(), columns=tfidf.get_feature_names_out())

(2226, 4577)

In [ ]:
log_reg_model = LogisticRegression(max_iter=1000)